In [1]:
import cv2,os

data_path=r'C:\Users\lakshay\Desktop\MACHINE_LEARNIG_REEMERGING\MASK DETECTION\train'
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories,labels)) #empty dictionary

print(label_dict)
print(categories)
print(labels)

{'without_mask': 0, 'with_mask': 1}
['without_mask', 'with_mask']
[0, 1]


In [2]:
img_size=100
data=[]
target=[]


for category in categories:
    folder_path=os.path.join(data_path,category)
    #img_names=os.listdir(folder_path)
    
    for img_name in os.listdir(folder_path):
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)

        try:
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)           
            #Coverting the image into gray scale
            resized=cv2.resize(gray,(img_size,img_size))
            #resizing the gray scale into 50x50, since we need a fixed common size for all the images in the dataset
            data.append(resized)
            target.append(label_dict[category])
            #appending the image and the label(categorized) into the list (dataset)

        except Exception as e:
            print('Exception:',e)
            #if any exception rasied, the exception will be printed here. And pass to the next image
            


In [3]:

import numpy as np

data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,1))
target=np.array(target)

from keras.utils import np_utils

new_target=np_utils.to_categorical(target)

In [5]:
np.save('data',data)
np.save('target',new_target)
data=np.load('data.npy')
target=np.load('target.npy')

#loading the save numpy arrays in the previous code

In [6]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint

model=Sequential()

model.add(Conv2D(200,(3,3),input_shape=data.shape[1:],padding = 'same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The first CNN layer followed by Relu and MaxPooling layers

model.add(Conv2D(100,(3,3),padding = 'same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The second convolution layer followed by Relu and MaxPooling layers

model.add(Flatten())
model.add(Dropout(0.5))
#Flatten layer to stack the output convolutions from second convolution layer
model.add(Dense(50,activation='relu'))
#Dense layer of 64 neurons
model.add(Dense(2,activation='softmax'))
#The Final layer with two outputs for two categories

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [7]:
from sklearn.model_selection import train_test_split

train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.1)

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 100, 100, 200)     2000      
_________________________________________________________________
activation (Activation)      (None, 100, 100, 200)     0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 50, 50, 200)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 50, 50, 100)       180100    
_________________________________________________________________
activation_1 (Activation)    (None, 50, 50, 100)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 25, 25, 100)       0         
_________________________________________________________________
flatten (Flatten)            (None, 62500)             0

In [10]:
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(train_data,train_target,epochs=4,callbacks=[checkpoint],validation_split=0.2)

Epoch 1/4
29/29 [==============================] - 39s 1s/step - loss: 0.6058 - accuracy: 0.6814 - val_loss: 0.5258 - val_accuracy: 0.7832
INFO:tensorflow:Assets written to: model-001.model\assets
Epoch 2/4
29/29 [==============================] - 40s 1s/step - loss: 0.4473 - accuracy: 0.7976 - val_loss: 0.3396 - val_accuracy: 0.8850
INFO:tensorflow:Assets written to: model-002.model\assets
Epoch 3/4
29/29 [==============================] - 37s 1s/step - loss: 0.3192 - accuracy: 0.8827 - val_loss: 0.2558 - val_accuracy: 0.9115
INFO:tensorflow:Assets written to: model-003.model\assets
Epoch 4/4
29/29 [==============================] - 37s 1s/step - loss: 0.2611 - accuracy: 0.8971 - val_loss: 0.3361 - val_accuracy: 0.8496


In [ ]:
from matplotlib import pyplot as plt

plt.plot(history.history['loss'],'r',label='training loss')
plt.plot(history.history['val_loss'],label='validation loss')
plt.xlabel('# epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['accuracy'],'r',label='training accuracy')
plt.plot(history.history['val_accuracy'],label='validation accuracy')
plt.xlabel('# epochs')
plt.ylabel('loss')
plt.legend()
plt.show()


In [11]:
print(model.evaluate(test_data,test_target))

4/4 [==============================] - 1s 357ms/step - loss: 0.3588 - accuracy: 0.8413
[0.35884177684783936, 0.841269850730896]


In [29]:
# image detect krlo pehla
import cv2 as cv

img = cv.imread(r'C:\Users\lakshay\Desktop\MACHINE_LEARNIG_REEMERGING\MASK DETECTION\test\with_mask\35-with-mask.jpg')
          

In [30]:
results={0:'without mask',1:'mask'}
img_size = 100
GR_dict={0:(0,0,255),1:(0,255,0)}
import cv2 as cv
import numpy as np
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)

haarcascade = cv.CascadeClassifier(cv.data.haarcascades + "haarcascade_frontalface_default.xml")
faces = haarcascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=1)

for (x,y,w,h) in faces:
    faces_roi = gray[y:y+h,x:x+w]
    
    resized=cv.resize(faces_roi,(img_size,img_size))
    d=np.array(resized)/255.0
    #print(d.shape)
    #data=np.array(faces_roi)/255.0
    #d.append(resized)
    data=np.reshape(d,(1,img_size,img_size,1))
    
    result=model.predict(data)
    
    label=np.argmax(result)
    
    cv.rectangle(img,(x,y),(x+w,y+h),GR_dict[label],2)
    cv.putText(img, results[label], (x, y-10),cv.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
cv.imshow("Detected Image",img)
cv.waitKey(0)
    

    #resized=cv2.resize(gray,(img_size,img_size))
            #resizing the gray scale into 50x50, since we need a fixed common size for all the images in the dataset
     
    #data.append(resized)


-1

In [3]:
#im saving this model as results are quite good
model.save('mask_project_new.h5')

In [2]:
import tensorflow as tf

model = tf.keras.models.load_model('mask_project_new.h5')

In [37]:
os.getcwd()

'C:\\Users\\lakshay'

In [6]:
results={0:'without mask',1:'mask'}
img_size = 100
GR_dict={0:(0,0,255),1:(0,255,0)}
import cv2 as cv
import numpy as np
capture = cv.VideoCapture(0)

istrue=True

while istrue:
    
    istrue,img = capture.read()
    
    if(istrue == False):
        break
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)

    haarcascade = cv.CascadeClassifier(cv.data.haarcascades + "haarcascade_frontalface_default.xml")
    faces = haarcascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5)

    for (x,y,w,h) in faces:
        faces_roi = gray[y:y+h,x:x+w]
    
        resized=cv.resize(faces_roi,(img_size,img_size))
        d=np.array(resized)/255.0
    #print(d.shape)
    #data=np.array(faces_roi)/255.0
    #d.append(resized)
        data=np.reshape(d,(1,img_size,img_size,1))
    
        result=model.predict(data)
    
        label=np.argmax(result)
    
        cv.rectangle(img,(x,y),(x+w,y+h),GR_dict[label],2)
        cv.putText(img, results[label], (x, y-10),cv.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
    cv.imshow("Detected Image",img)
    if(cv.waitKey(20) & 0xff == ord('d')):
        break
    
capture.release()
cv.destroyAllWindows()
    #cv.imshow("Video",frame)
    
    #gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
#cv.imshow("Gray",gray)
    #haar_cascade = cv.CascadeClassifier(cv.data.haarcascades + "haarcascade_frontalface_default.xml")
#haar_cascade = cv.CascadeClassifier('')


#faces_rect = haar_cascade.detectMultiScale(gray,scaleFactor = 0,minNeighbors = 0 )

    #faces_rect = haar_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=4);

    #for x,y,w,h in faces_rect:
        #cv.rectangle(img, (x,y),(x+w,y+h),(0,255,0),2)
    
    #cv.imshow("Detected faces",img)
    
    #if(cv.waitKey(20) & 0xff == ord('d')):
        #break
        
    
